In [10]:
from djitellopy import tello
import cv2
import HandTrackingModule as htm
import autopy #For controlling the mouse
import numpy as np
import time# Set resolution 

In [11]:
tello = tello.Tello()
tello.connect()
print(tello.get_battery())
tello.streamon() #打开摄像头

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


65


In [6]:
wCam, hCam = 1080, 720
frameR = 40
smoothening = 5
wScr, hScr = autopy.screen.size()  # 返回电脑屏幕的宽和高(1920.0, 1080.0)
#####################

cap = cv2.VideoCapture(0)  # Default camera on laptop: 0  External:1 or more
cap.set(3, wCam)  #Fix the window w/h ratio to 3:4
cap.set(4, hCam) #and set the 分辨率
pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0
frame = 0  # 初始化累计帧数
toggle = False
fly_flag = False
prev_state = [1, 1, 1, 1, 1]  # 初始化上一帧状态
current_state = [1, 1, 1, 1, 1]  # 初始化当前正状态

detector = htm.handDetector(maxHands = 1)

In [7]:
while True:
    photo = tello.get_frame_read().frame
    photo = cv2.resize(photo, (720, 480))
    cv2.imshow("Photo",photo)
    cv2.waitKey(1) #keep the window visible rather than shutting down instantly
    
    success, img_origin = cap.read() #按帧读取视频
                              #如果读取值是正确帧，success=True,如果读取到结尾，False
                              #img就是每一帧的图像，是个三维矩阵.在cv2里是BGR
    # 1. Detecting hand and return key points 坐标
    allhands,img = detector.findHands(img_origin, draw=True) #手部定位
    # 在图像窗口上创建一个矩形框，在该区域内移动鼠标
    cv2.rectangle(img, (frameR, frameR), (wCam - 4*frameR, hCam - 10*frameR), (0, 255, 0), 2,  cv2.FONT_HERSHEY_PLAIN)

    if len(allhands) != 0:
        x1, y1 = allhands[0]['lmList'][4][0:2]
        x2, y2 = allhands[0]['lmList'][8][0:2]
        x3, y3 = allhands[0]['lmList'][12][0:2]
        fingers = detector.fingersUp(img, allhands)

        if fingers == [0,1,0,0,0] and fly_flag == False:
            tello.takeoff() # Tello的起飞指令
            fly_flag = True

        elif fingers == [1,0,0,0,0] and fly_flag == True:
            tello.land() # Tello的降落指令
            fly_flag = False

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    # img_copy = img.copy()
    cv2.putText(img_origin, f'fps:{int(fps)}', [15, 25],
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2) #Display Frames per second 每秒传输帧数
    cv2.imshow("Image", img_origin)

    if cv2.waitKey(1) == ord('q'):
        break



# finger_drag.stop()
cap.release()
cv2.destroyAllWindows()


[INFO] tello.py - 437 - Send command: 'takeoff'
[INFO] tello.py - 461 - Response takeoff: 'ok'
[INFO] tello.py - 437 - Send command: 'land'
[INFO] tello.py - 461 - Response land: 'ok'


In [18]:
print(tello.get_battery())

62


In [19]:
tello.takeoff() # Tello的起飞指令

[INFO] tello.py - 437 - Send command: 'takeoff'
[WARNING] tello.py - 447 - Aborting command 'takeoff'. Did not receive a response after 20 seconds
[INFO] tello.py - 437 - Send command: 'takeoff'
[INFO] tello.py - 461 - Response takeoff: 'ok'


In [37]:
# tello.move_left(100) # Tello向左平移100厘米
tello.rotate_counter_clockwise(90) # Tello在同一高度顺时针旋转90度
# tello.move_forward(100) # Tello向前飞行100厘米

[INFO] tello.py - 437 - Send command: 'ccw 90'
[INFO] tello.py - 461 - Response ccw 90: 'ok'


In [ ]:
tello.move_forward(20) # Tello向前飞行100厘米

In [23]:
tello.flip_back()


[INFO] tello.py - 437 - Send command: 'flip b'
[INFO] tello.py - 461 - Response flip b: 'ok'


In [22]:
tello.flip('f')

[INFO] tello.py - 437 - Send command: 'flip f'
[INFO] tello.py - 461 - Response flip f: 'ok'


In [24]:
tello.land() # Tello的降落指令

[INFO] tello.py - 437 - Send command: 'land'
[INFO] tello.py - 461 - Response land: 'ok'
